In [10]:
from google.cloud import storage
import os

# アップロード先のバケット名
GCS_BUCKET_NAME = "default-bucket" 


In [11]:
# Change below if necessary
PROJECT = !gcloud config get-value project  # noqa: E999
PROJECT = PROJECT[0]
BUCKET = PROJECT
REGION = "us-central1"

%env PROJECT=$PROJECT
%env REGION=$REGION

env: PROJECT=qwiklabs-asl-01-0f9e59936044
env: REGION=us-central1


In [12]:
%%bash
gcloud config set project $PROJECT
gcloud config set ai/region $REGION

Updated property [core/project].
Updated property [ai/region].


In [13]:
# ローカルのCSVファイル名（前述のPythonコードで生成したもの）
LOCAL_CSV_FILE = "ai_score_dummy_data_full.csv"
    
# GCSバケットに保存する際のファイル名/パス
GCS_DESTINATION_PATH = "data/" + LOCAL_CSV_FILE

print(GCS_DESTINATION_PATH)

data/ai_score_dummy_data_full.csv


In [14]:
def create_bucket_if_not_exists(storage_client, bucket_name, location):
    """
    GCSバケットが存在しない場合、新しく作成します。
    """
    bucket = storage_client.bucket(bucket_name)

    if not bucket.exists():
        print(f"⚠️ バケット '{bucket_name}' が見つかりません。新規作成します...")
        try:
            # バケットを作成（ロケーションは必要に応じて変更してください）
            bucket.create(location=location)
            print(f"✅ バケット '{bucket_name}' をロケーション '{location}' に作成しました。")
            return True
        except Exception as e:
            print(f"❌ バケット作成中にエラーが発生しました: {e}")
            print("権限（Storage Admin ロールなど）を確認してください。")
            return False
    else:
        print(f"✅ バケット '{bucket_name}' は既に存在します。")
        return True

In [15]:
def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """
    ローカルファイルをGoogle Cloud Storageバケットにアップロードします。

    Args:
        bucket_name (str): ターゲットのGCSバケット名 (例: 'my-data-bucket-123')
        source_file_name (str): アップロードするローカルファイルのパス (例: 'balance_sheet_data.csv')
        destination_blob_name (str): GCSバケット内のファイルのパス/名前 (例: 'data/financials/balance_sheet_2025.csv')
    """
    
    # 1. GCSクライアントの初期化
    try:
        storage_client = storage.Client()
    except Exception as e:
        print("❌ エラー: Google Cloud認証に失敗しました。")
        print("環境変数 GOOGLE_APPLICATION_CREDENTIALS が設定されているか確認してください。")
        print(f"詳細: {e}")
        return
    
    # 2. バケットの確認と作成
    if not create_bucket_if_not_exists(storage_client, bucket_name, REGION): 
        return # バケット作成に失敗した場合は処理を終了

    # 2. バケットの取得
    bucket = storage_client.bucket(bucket_name)

    # 3. Blob（GCS上のファイル）の作成
    blob = bucket.blob(destination_blob_name)

    # 4. ファイルのアップロード
    try:
        blob.upload_from_filename(source_file_name)
        
        # 成功メッセージ
        print(f"✅ ファイル '{source_file_name}' を GCSバケット '{bucket_name}' の '{destination_blob_name}' に正常にアップロードしました。")
        print(f"GCSパス: gs://{bucket_name}/{destination_blob_name}")
        
    except FileNotFoundError:
        print(f"❌ エラー: ローカルファイル '{source_file_name}' が見つかりません。パスを確認してください。")
    except Exception as e:
        print(f"❌ GCSへのアップロード中にエラーが発生しました: {e}")



In [17]:
# GCSバケットにアップロード
upload_to_gcs(GCS_BUCKET_NAME, LOCAL_CSV_FILE, GCS_DESTINATION_PATH)

Forbidden: 403 GET https://storage.googleapis.com/storage/v1/b/default-bucket?fields=name&prettyPrint=false: 238790803769-compute@developer.gserviceaccount.com does not have storage.buckets.get access to the Google Cloud Storage bucket. Permission 'storage.buckets.get' denied on resource (or it may not exist).